In [1]:
import numpy as np
from scipy import optimize
from casadi import *
import time
import autograd

In [7]:
pip install time, autograd

Note: you may need to restart the kernel to use updated packages.


ERROR: Invalid requirement: 'time,'


In [2]:
N=47 #taille des tableaux 
dt=0.5
t0=23
k=0.01
h=0.05
b=1/500

In [3]:
Tm=18
T_in=Tm
TM=30
PM=5000
c_cr=1
c_pl=3/2

In [16]:
c=np.full(96, c_pl)
c[2:15]=c_cr
c[26:31]=c_cr
c[48:] = 0
print(c)

[1.5 1.5 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.5 1.5 1.5
 1.5 1.5 1.5 1.5 1.5 1.5 1.5 1.5 1.  1.  1.  1.  1.  1.5 1.5 1.5 1.5 1.5
 1.5 1.5 1.5 1.5 1.5 1.5 1.5 1.5 1.5 1.5 1.5 1.5 0.  0.  0.  0.  0.  0.
 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
 0.  0.  0.  0.  0.  0. ]


In [ ]:
def f(x):
    return np.dot(c,x)
    
def c_eq(x):
    c=np.zeros(2*N+2)
    c[N]=x[N]cd 
    c[N+1]= x[N+1] - T_in
    t=t0
    #Te= 4+ 8*np.exp(-(t-12)**2/40)
    #g=np.exp(-(k+h)*dt)*x[N+1] +(1-np.exp(-(k+h)*dt))(b*x[0] +h*Te)/(k+h)
    
    for i in range(N+2,2*N+2):
        Te= 4+ 8*np.exp(-(t-12)**2/40)
        g=np.exp(-(k+h)*dt)*x[i-1] +(1-np.exp(-(k+h)*dt))(b*x[i-1-(N+1)]+h*Te)/(k+h)
        c[i]=x[i]- g
        t+=dt
    return c

def c_in(x):
    u=np.zeros(2*N+2)
    v=np.zeros(2*N+2)
    cin=np.zeros((2*N+2,2))
    for i in range(N+1, 2*N+2):
        if (16+N+1<=i and i<=20+N+1):
            v[i]=Tm - x[i]
            u[i]=x[i]- TM
        else:
            if (38+N+1<=i and i<=47+N+1):
                v[i]=Tm - x[i]
                u[i]=x[i]- TM
    for i in range(0,N+1):
        u[i]=x[i]- PM
        v[i]= - x[i]
    cin[:,0]=u
    cin[:,1]=v
    return cin


In [ ]:
opti = casadi.Opti()
n = 2*N+2
x = opti.variable(n)
f=0
for i in range(n):
    f+=x[i]*c[i]

#u=(c_in(x))[:,0]
#v=(c_in(x))[:,1]
cont=c_in(x)
for i in range(n):
    u_i=cont[i,0]
    v_i=cont[i,1]
    
opti.minimize(f)
for i in range(n):
    opti.subject_to(u_i)
    opti.subject_to(v_i)
    
opti.subject_to(c_eq(x)==0)

x0 = np.zeros(2*N+2)
x[N+1]= T_in

opti.set_initial(x,x0)
opti.solver('ipopt')
sol = opti.solve()
print(sol.value(x))

In [ ]:
def grad(f):
    d = autograd.grad
    def grad_f(x, y):
        return np.array([d(f, 0)(x, y), d(f, 1)(x, y)])
    return grad_f

def Jac(f):
    j = autograd.jacobian
    def J_f(x, y):
        return np.array([j(f, 0)(x, y), j(f, 1)(x, y)]).T
    return J_f

In [ ]:
def f_min(x):
    np.dot(c,x)
    
grad_f_min=grad(f_min)
J_cin= Jac(c_in)
J_ceq= Jac(c_eq)

x0 = np.zeros(2*N+2)
x[N+1]= T_in
    
ineq_cons = {'type': 'ineq','fun' : lambda x: c_in(x),'jac' : lambda x: J_cin(x)}
eq_cons={'type': 'eq','fun' : lambda x: c_eq(x),'jac' : lambda x: J_ceq(x)}
#res = optimize.minimize(f, x0, method='SLSQP', jac=grad_f_min,
#constraints=[ineq_cons, eq_cons], options={'disp': True})

#print(res.x)

### Partie 3

In [ ]:
#Si on veut minimiser le coût total des nl logements:
def f(x):
    x_T=np.transpose(x)
    couts= np.matmul(x_T,c)
    return np.sum(couts)

In [2]:
Tm_1=18
Tm_2= 20
TM=30
PM=5000
c_cr=1
c_pl=3/2

In [3]:
#j=1 ou 2
#x est la concaténation des vecteurs associés à chaque logement; x est de taille (2N+2)*2= 4N+4, les 2N+2 premières coordonnées
#correspondent au logement 1 (j=1) -pressions et températures - et les 2N+2 dernières au logement 2 (j=2)

def c_eq_bis(x,j):
    
    indice_autre=2
    if j==2:
        indice_autre=1
    if j==1:
        T_in=Tm_1
    else:
        T_in=Tm_2
        
    c=np.zeros((2*N+2,2))
    c[N]=x[N + (2*N+2)*(j-1)]
    c[N+1]= x[N+1+(2*N+2)*j] - T_in
    t=t0
    #Te= 4+ 8*np.exp(-(t-12)**2/40)
    #g=np.exp(-(k+2*h)*dt)*x[N+1 +(2*N+2)*(j-1)]+(1-np.exp(-(k+2*h)*dt))(b*x[0 +(2*N+2)*j] +h*Te+h*T_in)/(k+2*h)
    
    for i in range(N+2,2*N+2):
        Te= 4+ 8*np.exp(-(t-12)**2/40)
        g=np.exp(-(k+ 2*h)*dt)*x[i-1 +(2*N+2)*(j-1)] +(1-np.exp(-(k+2*h)*dt))(b*x[i-1-(N+1) +(2*N+2)*(j-1)]+h*Te+h*x[i + (2*N+2)*(indice_autre -1)])/(k+2*h)
        c[i]=x[i +(2*N+2)*(j-1)]- g
        t+=dt
    return c

def c_in_bis(x):
    u1=np.zeros(2*N+2)
    v1=np.zeros(2*N+2)
    u2=np.zeros(2*N+2)
    v2=np.zeros(2*N+2)
    cin=np.zeros((2*N+2,4))
    for i in range(N+1, 2*N+2):
        if (16+N+1<=i and i<=20+N+1):
            v1[i]=Tm - x[i]
            u1[i]=x[i]- TM
            v2[i]=Tm - x[i+(2*N+2)]
            u2[i]=x[i+(2*N+2)]- TM
        else:
            if (38+N+1<=i and i<=47+N+1):
                v1[i]=Tm_1 - x[i][0]
                u1[i]=x[i][1]- TM
                v2[i]=Tm_2 - x[i+(2*N+2)][1]
                u2[i]=x[i+(2*N+2)][1]- TM
    for i in range(0,N+1):
        u1[i]=x[i]- PM
        v1[i]= - x[i]
        u2[i]=x[i+(2*N+2)]- PM
        v2[i]= - x[i+(2*N+2)]
    cin[:,0]= u1
    cin[:,0]= v1
    cin[:,0]= u2
    cin[:,0]= v2
    return cin